In [1]:
!pip install flask validators pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 1.8 MB/s eta 0:00:00


In [2]:
%%bash
cat > app.py <<'PY'
from flask import Flask, render_template, request, redirect
import sqlite3
import string
import random
import validators

app = Flask(__name__)

def get_db():
    conn = sqlite3.connect("urls.db")
    conn.row_factory = sqlite3.Row
    return conn

def create_table():
    conn = get_db()
    conn.execute("""
        CREATE TABLE IF NOT EXISTS urls (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            original_url TEXT NOT NULL,
            short_code TEXT UNIQUE NOT NULL
        )
    """)
    conn.commit()
    conn.close()

create_table()

def generate_short_code(length=6):
    chars = string.ascii_letters + string.digits
    return ''.join(random.choice(chars) for _ in range(length))

@app.route("/", methods=["GET", "POST"])
def index():
    short_url = None
    error = None

    if request.method == "POST":
        original_url = request.form["url"]

        if not validators.url(original_url):
            error = "Invalid URL. Please enter a valid URL."
        else:
            short_code = generate_short_code()

            conn = get_db()
            conn.execute(
                "INSERT INTO urls (original_url, short_code) VALUES (?, ?)",
                (original_url, short_code)
            )
            conn.commit()
            conn.close()

            short_url = request.host_url + short_code

    return render_template("index.html", short_url=short_url, error=error)

@app.route("/<short_code>")
def redirect_url(short_code):
    conn = get_db()
    result = conn.execute(
        "SELECT original_url FROM urls WHERE short_code = ?",
        (short_code,)
    ).fetchone()
    conn.close()

    if result:
        return redirect(result["original_url"])
    else:
        return "URL Not Found", 404

@app.route("/history")
def history():
    conn = get_db()
    urls = conn.execute("SELECT * FROM urls").fetchall()
    conn.close()
    return render_template("history.html", urls=urls)

if __name__ == "__main__":
    app.run(debug=True)
PY

In [3]:
%%bash
mkdir -p templates
cat > templates/index.html <<'HTML'
<!DOCTYPE html>
<html>
<head>
    <title>URL Shortener</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
</head>
<body class="container mt-5">

<h2>URL Shortener</h2>

<form method="POST">
    <input type="text" name="url" class="form-control" placeholder="Enter URL" required>
    <button type="submit" class="btn btn-primary mt-3">Shorten URL</button>
</form>

{% if error %}
<div class="alert alert-danger mt-3">{{ error }}</div>
{% endif %}

{% if short_url %}
<div class="mt-3">
    <input type="text" id="shortUrl" class="form-control" value="{{ short_url }}" readonly>
    <button onclick="copyText()" class="btn btn-success mt-2">Copy</button>
</div>
{% endif %}

<a href="/history" class="btn btn-link mt-4">View History</a>

<script>
function copyText() {
    let copyInput = document.getElementById("shortUrl");
    copyInput.select();
    copyInput.setSelectionRange(0, 99999);
    document.execCommand("copy");
    alert("Copied!");
}
</script>

</body>
</html>
HTML

In [4]:
%%bash
cat > templates/history.html <<'HTML'
<!DOCTYPE html>
<html>
<head>
    <title>History</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
</head>
<body class="container mt-5">

<h2>URL History</h2>

<table class="table table-bordered">
    <thead>
        <tr>
            <th>Original URL</th>
            <th>Short URL</th>
        </tr>
    </thead>
    <tbody>
        {% for url in urls %}
        <tr>
            <td>{{ url["original_url"] }}</td>
            <td>{{ request.host_url }}{{ url["short_code"] }}</td>
        </tr>
        {% endfor %}
    </tbody>
</table>

<a href="/" class="btn btn-primary">Back</a>

</body>
</html>
HTML

In [13]:
from pyngrok import ngrok
import subprocess

# Set your ngrok authtoken here. Replace 'YOUR_NGROK_AUTH_TOKEN' with your actual token.
# You can get one from https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("30sDu6MUIz0Xc5jMBAa86fYKwol_5YiYi8VZ35sQgeS1FCoCy")

# Start ngrok tunnel
public_url = ngrok.connect(5000).public_url
print("Public URL:", public_url)

# Run Flask app
subprocess.Popen(["python", "app.py"])

Public URL: https://3a817f0d6b9f.ngrok-free.app


<Popen: returncode: None args: ['python', 'app.py']>